# Transformermodellen for prediksjon av sekvenser

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

OPPGAVE 1.1

For å trene en transformermodell så trenger vi et datasett $\{x, y\}$ som består at flere x og y vektorer, $ D = \{ (x_0, y_0), (x_1, y_1), \ldots, (x_{n-1}, y_{n-1}) \} $. For å vise et eksempel velger vi $r = 2$, $a = 43$, $b = 7$, $c = 18$ og $d = a \cdot b + c = 319$. Vektoren $x_i$ er bygget opp slik at vi får de $r$ første sifferene av $a$, $b$, $c$ og $d$ og lengden til $x_i$ blir dermed $4 \cdot r$. I dette tilfellet får vi da $x_0 = [4, 3, 0, 7, 8, 3, 1]$. $y_i$ 




OPPGAVE 2
Når optimeringen er ferdig, hvordan kan vi bruke modellen fθ til å predikere d gitt
a, b, c? Vis dette med et eksempel, på samme måte som i likning 11

Vi skal  nå anta at optimeringen er ferdig og at vi har funnet parametrene ${\theta}$ som minimerer objektfunksjonen $\mathscr{L}$, vi skal dermed bruke modellen $f_{\theta}$ for å predikere $d$ gitt $a, b$ og $c$. $f_{\theta}$ skal testes på en sekvens som bare inneholder tallene til $a, b,$ og $c$ og vi befinner oss i testfasen. Vi vil ha $x \in \mathbb{Z^{2r}}$ som gir oss $\hat{z} = f_{\theta}(x)$ der $\hat{z}\in \mathbb{Z^{2r}}$.


Vi har fortsatt $r = 2$, $a = 43$, $b = 7$ og $c = 18$


Hver evaluering av modellen predikerer neste sifferet i sekvensen  og legger det til siste predikerte heltatt i $x$ for å bruke dette som input i neste prediksjon helt til det er predikert $r+1$ nye heltall som ideelt sett skal være lik $d$. 


Vi ønsker å finne ${\theta}$ slik at $\hat{y} = [\hat{z_5}, \hat{z_6}, \hat{z_7}] = [3, 1, 9]$

Vi starter med $x^{(0)} = [4,3,0,7,1,8]$ og kjører den gjennom modellen $f_{\theta}$

$f_{\theta}([4,3,0,7,1,8]) = [\hat{z_0}^{(0)}, \hat{z_1}^{(0)},..., \hat{z_5}^{(0)}]$ 

Neste sekvens blir dermed:
$x^{(1)} = [4,3,0,7,1,8, \hat{z_5}^{(0)}]$

Følger samme prosedyre en gang til:

$f_{\theta}([4,3,0,7,1,8, \hat{z_5}^{(0)}]) = [\hat{z_0}^{(1)}, \hat{z_1}^{(1)},..., \hat{z_6}^{(1)}]$ 

Neste sekvens blir:

$x^{(2)} = [4,3,0,7,1,8, \hat{z_5}^{(0)}, \hat{z_6}^{(1)}]$

Putter $x^{(2)}$ inni modellen:

$f_{\theta}([4,3,0,7,1,8, \hat{z_5}^{(0)}], \hat{z_6}^{(1)}) = [\hat{z_0}^{(2)}, \hat{z_1}^{(2)},..., \hat{z_7}^{(2)}]$ 

Og ender tilslutt opp med sekvensen:

$x^{(3)} = [4,3,0,7,1,8, \hat{z_5}^{(0)}, \hat{z_6}^{(1)}, \hat{z_7}^{(2)}]$

De $r-1=3$ siste elementene i sekvensen $x^{(3)}$ skal da tilsvare vår $\hat{y} = [3,1,9]$
